# dbApps09 Walkthrough: Security & Transactions

**Course:** Database Applications Development (145085), Medina County Career Center

## Setup: Import Libraries & Create Demo Database

In [ ]:
import pandas as pd
import sqlite3

# Create a new in-memory database for demos
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

print("Database connection established.")

## Create Users Table with Sample Data

In [ ]:
# Create users table with columns: userId, username, password, email
createUsersSQL = """
CREATE TABLE users (
    userId INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT NOT NULL,
    password TEXT NOT NULL,
    email TEXT NOT NULL
)
"""

conn.execute(createUsersSQL)
conn.commit()

print("users table created.")

In [ ]:
# Insert 5 sample user records
sampleUsers = [
    ("admin", "password123", "admin@company.com"),
    ("alice", "secret456", "alice@company.com"),
    ("bob", "bobpass789", "bob@company.com"),
    ("charlie", "char999", "charlie@company.com"),
    ("diana", "diana2024", "diana@company.com")
]

insertUsersSQL = "INSERT INTO users (username, password, email) VALUES (?, ?, ?)"
conn.executemany(insertUsersSQL, sampleUsers)
conn.commit()

print("5 sample users inserted.")

In [ ]:
# Verify users table contents
dfUsers = pd.read_sql("SELECT * FROM users", conn)
print(dfUsers)

---

# Sub-Lesson 09a — SQL Injection & Parameterized Queries

SQL Injection is a security vulnerability where malicious SQL code is inserted into input fields. This can allow attackers to:
- Read unauthorized data
- Modify or delete data
- Execute arbitrary commands

The solution is **parameterized queries** (also called prepared statements).

## The VULNERABLE Approach: String Concatenation

In [ ]:
# VULNERABLE: Building SQL with string concatenation
# DO NOT use this approach in production!

userInput = "admin"

# This constructs a SQL query by concatenating strings
vulnerableQuery = f"SELECT * FROM users WHERE username = '{userInput}'"

print("Query string:")
print(vulnerableQuery)
print()

# When userInput = "admin", this works as intended
dfResult = pd.read_sql(vulnerableQuery, conn)
print("Result with userInput = 'admin':")
print(dfResult)

## The Problem: Malicious Input

What happens if the user input contains SQL syntax?

### Example 1: Return ALL rows

If `userInput = ' OR '1'='1` , the constructed query becomes:

```sql
SELECT * FROM users WHERE username = '' OR '1'='1'
```

The condition `'1'='1'` is ALWAYS TRUE, so it returns ALL rows in the table, bypassing authentication!

### Example 2: Delete Data

If `userInput = '; DROP TABLE users; --` , the constructed query becomes:

```sql
SELECT * FROM users WHERE username = ''; DROP TABLE users; --'
```

The `--` is a SQL comment that ignores the rest of the line. This would execute the `DROP TABLE` command and destroy the entire users table!

### Example 3: Insert Malicious Data

If `userInput = ' OR '1'='1'; INSERT INTO users (username, password, email) VALUES ('hacker', 'hack', 'hacker@evil.com'); --` , malicious records could be added to the database.

## The SAFE Approach: Parameterized Queries

Use the `?` placeholder for each parameter. The database driver handles escaping and ensures the input is treated as DATA, not as SQL code.

In [ ]:
# SAFE: Parameterized query with placeholder ?

userInput = "admin"

# The ? is a placeholder for the parameter
safeQuery = "SELECT * FROM users WHERE username = ?"

print("Query template:")
print(safeQuery)
print()

# Pass the parameter separately as a tuple
dfResult = pd.read_sql(safeQuery, conn, params=(userInput,))

print("Result with userInput = 'admin':")
print(dfResult)

In [ ]:
# Now try with malicious input using the SAFE approach

maliciousInput = "' OR '1'='1"

safeQuery = "SELECT * FROM users WHERE username = ?"

# The ? placeholder treats this as literal data, not SQL code
dfResult = pd.read_sql(safeQuery, conn, params=(maliciousInput,))

print("Result with userInput = \"' OR '1'='1\":")
print(dfResult)
print()
print("Notice: No rows returned. The input is treated as a literal string, not SQL syntax.")

## Why Parameterized Queries Are Safe

1. **Separation of Code and Data:** The SQL template (code) is separate from the parameters (data).
2. **Automatic Escaping:** The database driver escapes special characters so they are treated as literal values.
3. **Type Safety:** The driver knows the parameter is a string, so it won't interpret SQL keywords.

## Parameterized Queries with Multiple Parameters

## Try This: Write a Parameterized Query

Write a safe parameterized query to find a user by email address. Use the variable `targetEmail = "bob@company.com"` as the parameter.

In [ ]:
# TODO: Write a parameterized query to find a user by email
# Use targetEmail = "bob@company.com"
# Print the result

targetEmail = "bob@company.com"

# Your parameterized query here:



---

# Sub-Lesson 09b — Transactions & ACID

A **transaction** is a sequence of database operations that are treated as a single unit of work. Either ALL operations succeed, or NONE of them do. This ensures data consistency and integrity.

## ACID Properties

- **Atomicity:** A transaction is all-or-nothing. Either all statements succeed and are saved, or all are rolled back.
- **Consistency:** The database moves from one valid state to another valid state.
- **Isolation:** Concurrent transactions don't interfere with each other.
- **Durability:** Once committed, data persists even if the system crashes.

## Create Bank Accounts Table

In [ ]:
# Create a bank_accounts table for transaction examples
createBankAccountsSQL = """
CREATE TABLE bank_accounts (
    accountId INTEGER PRIMARY KEY AUTOINCREMENT,
    accountName TEXT NOT NULL,
    balance REAL NOT NULL
)
"""

conn.execute(createBankAccountsSQL)
conn.commit()

print("bank_accounts table created.")

In [ ]:
# Insert sample bank accounts
sampleAccounts = [
    ("Alice Checking", 1000.00),
    ("Bob Savings", 5000.00),
    ("Charlie Checking", 500.00)
]

insertAccountsSQL = "INSERT INTO bank_accounts (accountName, balance) VALUES (?, ?)"
conn.executemany(insertAccountsSQL, sampleAccounts)
conn.commit()

print("Sample accounts inserted.")

In [ ]:
# View all bank accounts
dfAccounts = pd.read_sql("SELECT * FROM bank_accounts", conn)
print(dfAccounts)

## Example: COMMIT a Transaction

Scenario: Transfer $100 from Alice's account (accountId 1) to Bob's account (accountId 2).

This is a single logical operation that requires TWO database updates:
1. Deduct $100 from Alice
2. Add $100 to Bob

Both must succeed together, or neither should happen.

In [ ]:
# BEGIN a transaction
conn.execute("BEGIN")

# Step 1: Deduct $100 from Alice's account (accountId 1)
conn.execute(
    "UPDATE bank_accounts SET balance = balance - 100 WHERE accountId = 1"
)

# Step 2: Add $100 to Bob's account (accountId 2)
conn.execute(
    "UPDATE bank_accounts SET balance = balance + 100 WHERE accountId = 2"
)

# COMMIT the transaction (save all changes)
conn.commit()

print("Transaction committed. Money transferred.")

In [ ]:
# Verify the transfer
dfAccounts = pd.read_sql("SELECT * FROM bank_accounts", conn)
print(dfAccounts)
print()
print("Alice's balance: $900 (was $1000)")
print("Bob's balance: $5100 (was $5000)")

## Example: ROLLBACK a Transaction

Scenario: Attempt to transfer $1000 from Charlie's account (accountId 3, balance $500) to Alice.

Since Charlie doesn't have enough funds, we detect the error BEFORE committing and ROLLBACK the entire transaction.

In [ ]:
# View Charlie's current balance
dfCharlie = pd.read_sql(
    "SELECT * FROM bank_accounts WHERE accountId = 3",
    conn
)
print("Charlie's account before transaction:")
print(dfCharlie)

In [ ]:
# BEGIN a transaction
conn.execute("BEGIN")

# Step 1: Deduct $1000 from Charlie's account (accountId 3)
# NOTE: Charlie only has $500, so this violates business logic
conn.execute(
    "UPDATE bank_accounts SET balance = balance - 1000 WHERE accountId = 3"
)

# Check Charlie's balance after the update (but before commit)
dfCheck = pd.read_sql(
    "SELECT accountId, accountName, balance FROM bank_accounts WHERE accountId = 3",
    conn
)
print("Charlie's balance after update (before commit):")
print(dfCheck)
print()
print("PROBLEM: Balance is negative! This violates business rules.")
print()

# Since the transfer violates business logic, ROLLBACK the entire transaction
conn.execute("ROLLBACK")
print("Transaction rolled back. Changes discarded.")

In [ ]:
# Verify that Charlie's balance is restored
dfCharlie = pd.read_sql(
    "SELECT * FROM bank_accounts WHERE accountId = 3",
    conn
)
print("Charlie's account after rollback:")
print(dfCharlie)
print()
print("Balance restored to $500 (original amount).")

## ACID Properties Explained with Bank Transfer Example

### Atomicity (All-or-Nothing)
The transfer of $100 from Alice to Bob is atomic. Either both updates complete, or neither happens. There's no intermediate state where Alice loses $100 but Bob never receives it.

### Consistency (Valid State)
Before the transfer: Alice=$1000, Bob=$5000, Total=$6000
After the transfer: Alice=$900, Bob=$5100, Total=$6000
The total money is conserved. The database moves from a valid state to another valid state.

### Isolation (No Interference)
If Alice and Charlie both initiate transfers at the same time, the database ensures they don't read or overwrite each other's changes. Each transaction works with a consistent view of the data.

### Durability (Persistent)
Once the transaction is committed, the data is written to disk. Even if the system crashes immediately after, the changes are preserved.

## Data Privacy Awareness

Databases often contain sensitive personal or protected information:

- **HIPAA (Health Insurance Portability and Accountability Act):** Protects patient health information in healthcare databases.
- **FERPA (Family Educational Rights and Privacy Act):** Protects student educational records in school databases.
- **PCI DSS (Payment Card Industry Data Security Standard):** Protects credit card and payment information.
- **GDPR (General Data Protection Regulation):** Protects personal data of EU residents.

When designing and securing databases, always consider:
1. What types of sensitive data are stored?
2. Who has access to this data?
3. Are SQL queries parameterized to prevent injection?
4. Are transactions used to maintain data consistency?
5. Are backups secure and regularly tested?
6. Are database logs monitored for unauthorized access attempts?

## Try This: Write a Transaction

Write a transaction that transfers $200 from Bob's account (accountId 2) to Charlie's account (accountId 3). Include:
1. BEGIN statement
2. Two UPDATE statements (debit Bob, credit Charlie)
3. COMMIT statement
4. A SELECT to verify the new balances

Before you write the code, what should Bob's new balance be? What should Charlie's be?

In [ ]:
# TODO: Write a transaction to transfer $200 from Bob to Charlie
# Bob is accountId 2, Charlie is accountId 3

# Your transaction code here:



## Summary: Key Takeaways

1. **Always use parameterized queries** to prevent SQL injection attacks. Never concatenate user input into SQL strings.
2. **Use transactions** for multi-step operations that must succeed together or fail together.
3. **COMMIT** when all operations succeed and the database should be updated.
4. **ROLLBACK** if an error occurs or business logic is violated.
5. **Understand ACID properties** to design reliable, consistent database systems.
6. **Protect sensitive data** by following HIPAA, FERPA, PCI DSS, GDPR, and other compliance standards.
7. **Log and monitor** database access to detect unauthorized activity.